This workbook contains code snippets used to answer questions in first on-line quiz.  There are few comments and no markers between questions so you have a bit of figuring out to do :-)


In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

from IPython.display import display, Markdown
pd.set_option('display.max_columns', None)  
DEBUG = False

import os

## Load churn.csv

In [138]:
df_churn = pd.read_csv("data/churn.csv")
df_churn.shape

(3333, 16)

In [139]:
df_churn.head()

,State,Account_Length,Area_Code,Intl_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Eve_Mins,Eve_Calls,Night_Mins,Night_Calls,Intl_Mins,Intl_Calls,Cust_Serv_Calls,Churn
0,KS,128,415,no,yes,25,265.1,110,197.4,99,244.7,91,10.0,3,1,no
1,OH,107,415,no,yes,26,161.6,123,195.5,103,254.4,103,13.7,3,1,no
2,NJ,137,415,no,no,0,243.4,114,121.2,110,162.6,104,12.2,5,0,no
3,OH,84,408,yes,no,0,299.4,71,61.9,88,196.9,89,6.6,7,2,no
4,OK,75,415,yes,no,0,166.7,113,148.3,122,186.9,121,10.1,3,3,no


## Load states.csv

In [140]:
df_states = pd.read_csv("data/states.csv")
df_states.shape

(52, 4)

In [141]:
df_states.head()

,State,Latitude,Longitude,Name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [142]:
df = df_churn.merge(df_states, on="State")
df.shape

(3333, 19)

## Questions

In [143]:
df['Account_Length'].max()

243

In [144]:
criteria = df.Churn=="yes"
print(criteria.value_counts())

df.loc[criteria].Account_Length.mean()

False    2850
True      483
Name: Churn, dtype: int64


102.66459627329192

In [145]:
type(df.loc[criteria, "Account_Length"])

pandas.core.series.Series

In [146]:
criteria = df.Cust_Serv_Calls==0
print(criteria.value_counts())

False    2636
True      697
Name: Cust_Serv_Calls, dtype: int64


In [147]:
100 * criteria.sum() / df.shape[0]

20.912091209120913

In [148]:
df.Account_Length.values[0:5]

array([128,  70,  92, 137, 132])

In [149]:
criteria = df.VMail_Plan=='yes'
print(criteria.value_counts())
df.loc[criteria].VMail_Message.mean()

False    2411
True      922
Name: VMail_Plan, dtype: int64


29.27765726681128

In [150]:
criteria = (df.VMail_Plan=='yes') & (df.Intl_Plan=='yes')
print(criteria.value_counts())
criteria.sum()

False    3241
True       92
dtype: int64


92

In [151]:
df.groupby("Churn")["Account_Length"].mean()

Churn
no     100.793684
yes    102.664596
Name: Account_Length, dtype: float64

In [152]:
df.groupby("State")["Account_Length"].mean().sort_values(ascending=False).head()

State
FL    109.571429
OK    108.262295
LA    108.235294
KS    106.785714
ND    106.209677
Name: Account_Length, dtype: float64

In [153]:
df.groupby("State")["Account_Length"].mean().sort_values(ascending=False).values[2]

108.23529411764706

In [154]:
df.groupby("State")["Account_Length"].mean().sort_values(ascending=False).index[2]

'LA'

In [155]:
df.groupby("State")[["Account_Length"]].mean().sort_values("Account_Length").head(5)

,Account_Length
State,
KY,91.830508
MA,91.861538
MT,92.676471
TN,92.716981
IA,94.318182


In [156]:
names = df.groupby("Name")['Account_Length'].median().sort_values(ascending=False).index[0:5]

In [157]:
names = set(names)
names

{'Florida', 'Kansas', 'Louisiana', 'North Carolina', 'Virginia'}

In [158]:
criteria = df.Name.isin(names)
print(criteria.value_counts())
df.loc[criteria, "Account_Length"].mean()

False    3004
True      329
Name: Name, dtype: int64


106.93009118541033

In [163]:
df['Churn_Flag'] = df.Churn.map( {'yes':True, 'no':False})

In [164]:
df.Churn_Flag.value_counts()

False    2850
True      483
Name: Churn_Flag, dtype: int64

In [170]:
df.groupby("Name")['Churn_Flag'].agg(sum).head(2)

Name
Alabama    8.0
Alaska     3.0
Name: Churn_Flag, dtype: float64

In [171]:
df.groupby("Name")['Churn_Flag'].agg("sum").head(2)

Name
Alabama    8.0
Alaska     3.0
Name: Churn_Flag, dtype: float64

In [172]:
df.groupby("Name")['Churn_Flag'].sum().head(2)

Name
Alabama    8.0
Alaska     3.0
Name: Churn_Flag, dtype: float64

In [173]:
df.groupby("Name")['Churn_Flag'].agg([sum]).head(2)

,sum
Name,
Alabama,8.0
Alaska,3.0


In [177]:
df_tmp = df.groupby("Name")['Churn_Flag'].agg(["sum", "count"])

In [184]:
df_tmp['Churn_Rate'] = df_tmp['sum'] / df_tmp['count']

In [185]:
criteria = df_tmp.Churn_Rate>=0.20
print(criteria.value_counts())

False    39
True     12
Name: Churn_Rate, dtype: int64


In [186]:
df2 = pd.crosstab(df.Name, df.Churn, margins=True,normalize='index').sort_values("yes", ascending=False)
result = df2.loc[df2.yes>=0.2].shape[0]
display(result)

12

In [187]:
states = set(df.groupby("Name")["State"].count().sort_values(ascending=False).head(20).index)
states

{'Alabama',
 'Connecticut',
 'Idaho',
 'Indiana',
 'Kansas',
 'Maryland',
 'Michigan',
 'Minnesota',
 'New Jersey',
 'New York',
 'North Carolina',
 'Ohio',
 'Oregon',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'West Virginia',
 'Wisconsin',
 'Wyoming'}

In [189]:
criteria = df.Name.isin(states)
print(criteria.value_counts())
result_a = df.loc[criteria].groupby("Name")["Account_Length"].mean()
result_a

False    1808
True     1525
Name: Name, dtype: int64


Name
Alabama            98.025000
Connecticut        99.040541
Idaho             104.493151
Indiana           100.183099
Kansas            106.785714
Maryland          105.271429
Michigan           98.109589
Minnesota         101.571429
New Jersey        100.205882
New York          100.698795
North Carolina    104.779412
Ohio              102.153846
Oregon            101.166667
Texas              97.541667
Utah              100.958333
Vermont           104.424658
Virginia          105.935065
West Virginia      95.433962
Wisconsin          98.884615
Wyoming           105.740260
Name: Account_Length, dtype: float64

In [190]:
result_max = result_a.max()
result_min = result_a.min()
result = result_max - result_min
display(result)

11.351752021563343

['Categorical',
 'CategoricalDtype',
 'CategoricalIndex',
 'DataFrame',
 'DateOffset',
 'DatetimeIndex',
 'DatetimeTZDtype',
 'ExcelFile',
 'ExcelWriter',
 'Float64Index',
 'Grouper',
 'HDFStore',
 'Index',
 'IndexSlice',
 'Int16Dtype',
 'Int32Dtype',
 'Int64Dtype',
 'Int64Index',
 'Int8Dtype',
 'Interval',
 'IntervalDtype',
 'IntervalIndex',
 'MultiIndex',
 'NaT',
 'Panel',
 'Period',
 'PeriodDtype',
 'PeriodIndex',
 'RangeIndex',
 'Series',
 'SparseArray',
 'SparseDataFrame',
 'SparseDtype',
 'SparseSeries',
 'TimeGrouper',
 'Timedelta',
 'TimedeltaIndex',
 'Timestamp',
 'UInt16Dtype',
 'UInt32Dtype',
 'UInt64Dtype',
 'UInt64Index',
 'UInt8Dtype',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__docformat__',
 '__file__',
 '__git_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_hashtable',
 '_lib',
 '_libs',
 '_np_version_under1p13',
 '_np_version_under1p14',
 '_np_version_under1p15',
 '_np_version_under1p16',
 '_np_version_under1p17